In [2]:
!pip install fiona

In [3]:
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt

In [4]:
Primary_school_path = r"E:\GIS_File\Project\Geo-project\1st\Primary School.gdb\Primary School.gdb"
layers = fiona.listlayers(Primary_school_path)
layers

['Primary_school_point', 'Road', 'Ward_boundary', 'River']

In [6]:
# Load the ward boundaries
wards = gpd.read_file(Primary_school_path, layer="Ward_boundary")

In [7]:
print(wards.crs)

PROJCS["BUTM",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",90],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [9]:
# Combine all ward polygons to get the municipal boundary
municipal_boundary = wards.unary_union

C:\Users\HP\AppData\Local\Temp\ipykernel_11184\773326570.py:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  municipal_boundary = wards.unary_union


In [11]:
# Convert to GeoDataFrame
municipal_boundary = gpd.GeoDataFrame(geometry=[municipal_boundary], crs=wards.crs)

In [12]:
municipal_boundary

,geometry
0,"POLYGON ((441611.903 2542953.353, 441622.994 2..."


In [15]:
# Step 2: Clip buffer with municipal boundary to limit the area
buffer_clipped = gpd.overlay(schools_buffer, municipal_boundary, how='intersection')

In [14]:
# Step 1: Create 500m buffer around each school
schools_buffer = Primary_school.copy()
schools_buffer['geometry'] = schools_buffer.buffer(500)

In [16]:
# Step 3: Union all buffer zones together
all_buffers_union = buffer_clipped.unary_union

C:\Users\HP\AppData\Local\Temp\ipykernel_11184\1536890695.py:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  all_buffers_union = buffer_clipped.unary_union


In [23]:
# Step 4: Served area = area covered by buffer
served_area = gpd.GeoDataFrame(geometry=[all_buffers_union], crs=municipal_boundary.crs)

In [24]:
from shapely.geometry import Polygon
from shapely.ops import unary_union
import itertools

# Step 5: Overserved area = overlap between different school buffers
overserved_geoms = []

for i, j in itertools.combinations(schools_buffer.geometry, 2):
    inter = i.intersection(j)
    if not inter.is_empty and inter.area > 0:
        overserved_geoms.append(inter)

# Convert to GeoDataFrame
if overserved_geoms:
    overserved_union = unary_union(overserved_geoms)
    overserved_gdf = gpd.GeoDataFrame(geometry=[overserved_union], crs=municipal_boundary.crs)

In [26]:
# Step 6: Underserved area = total municipal area - buffer area
underserved = gpd.overlay(municipal_boundary, served_area, how='difference')

In [27]:
underserved

,geometry
0,"MULTIPOLYGON (((441622.994 2542920.082, 441576..."


In [28]:
# Step 7: Calculate areas (in square meters → km²)
total_area = municipal_boundary.geometry.area.sum() / 1e6
served_area_km2 = served_area.geometry.area.sum() / 1e6
overserved_area_km2 = overserved_gdf.geometry.area.sum() / 1e6
underserved_area_km2 = underserved.geometry.area.sum() / 1e6

In [29]:
# Step 8: Calculate percentages
served_percent = (served_area_km2 / total_area) * 100
overserved_percent = (overserved_area_km2 / total_area) * 100
underserved_percent = (underserved_area_km2 / total_area) * 100

In [30]:
# Print results
print("Total Area (km²):", round(total_area, 2))
print("Served Area:", round(served_area_km2, 2), "km² (", round(served_percent, 2), "% )")
print("Overserved Area:", round(overserved_area_km2, 2), "km² (", round(overserved_percent, 2), "% )")
print("Underserved Area:", round(underserved_area_km2, 2), "km² (", round(underserved_percent, 2), "% )")

Total Area (km²): 27.43
Served Area: 9.51 km² ( 34.68 % )
Overserved Area: 0.79 km² ( 2.87 % )
Underserved Area: 17.92 km² ( 65.32 % )


In [36]:
import leafmap.foliumap as leafmap

# Initialize the map centered on the municipality
m = leafmap.Map(center=[22.8, 89.55], zoom=14)  # Adjust lat/lon if needed

# Add layers
m.add_gdf(municipal_boundary, layer_name="Municipal_Boundary", style={"fillOpacity": .3, "color": "black"})
m.add_gdf(served_area, layer_name="Served Area", style={"color": "green", "fillOpacity": 0.4})
m.add_gdf(Primary_school, layer_name="Primary Schools", marker_type="circle", marker_kwargs={"radius": 5, "color": "red"})
m.add_gdf(underserved, layer_name="underserved", style={"color": "pink", "fillOpacity": 0.4})
m.add_gdf(overserved_gdf, layer_name="overserved", style={"color": "green", "fillOpacity": 0.4})
m

In [33]:
import geopandas as gpd
import leafmap.foliumap as leafmap


In [34]:
# Ensure consistent CRS
wards = wards.to_crs(epsg=4326)
Primary_school = Primary_school.to_crs(epsg=4326)
served_area = served_area.to_crs(epsg=4326)

# Calculate served area % by ward
wards['served_area'] = wards.geometry.intersection(served_area.unary_union).area
wards['served_percent'] = (wards['served_area'] / wards.geometry.area) * 100

# Identify underserved wards (<50% served)
underserved_wards = wards[wards['served_percent'] < 50].copy()

# Proposed school centroids (top 3 most underserved)
underserved_wards_sorted = underserved_wards.sort_values('served_percent')
proposed_points = underserved_wards_sorted.head(3).copy()
proposed_points['geometry'] = proposed_points.centroid

# Initialize interactive map
m = leafmap.Map(center=[22.8, 89.6], zoom=13)
m.add_basemap("CartoDB.Positron")

# Add ward layer colored by served percent
m.add_gdf(
    wards,
    layer_name="Wards by Served %",
    info_mode="on_hover",
    color_column="served_percent",
    cmap="black",
    style={"fillOpacity": 0.6}
)

# Add existing school points
m.add_gdf(Primary_school, layer_name="Existing Schools", icon_data="school", marker_color="blue")

# Add proposed school points (red star markers)
m.add_gdf(
    proposed_points,
    layer_name="Proposed Schools",
    marker_type="star",
    marker_color="red",
    marker_size=15
)


# Show map in notebook (optional)
m

In [38]:
# Save as interactive HTML
m.save("HTML/primary_school_service_webmap2.html")